### Обязательно к первоочердному выполнению

In [ ]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

In [123]:
import csv
from datetime import date, timedelta

registration_date_dict = {}
registration_id_dict = {}
with open('registrations.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    header = next(reader)
    for user_id, entry_date in reader:
        registration_date_dict.setdefault(date.fromisoformat(entry_date), set()).add(int(user_id))
        registration_id_dict[int(user_id)] = date.fromisoformat(entry_date)

entries_date_dict = {}
entries_id_dict = {}
with open('entries.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    header = next(reader)
    for user_id, entry_date in reader:
        entries_date_dict.setdefault(date.fromisoformat(entry_date), set()).add(int(user_id))
        entries_id_dict[int(user_id)] = date.fromisoformat(entry_date)

answers = []
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

### Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе

In [124]:
cohort_start_date = date(2021, 1, 1)
cohort_end_date = date(2021, 2, 1)
registrations_qty = 0
entries_qty = 0

for k, v in registration_date_dict.items():
    if k >= cohort_start_date and k < cohort_end_date:
        registrations_qty += len(v)
        entries_qty += len(v.intersection(entries_date_dict.get(k +  timedelta(days=15), {})))

retention_15_day = round((entries_qty/registrations_qty)*100, 5)

**Проверка:**

In [125]:
correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Rolling-retention 30 дня (в процентах) для пользователей, зарегистрированных в январе

In [ ]:
cohort_dict = {key: value for key, value in registration_id_dict.items() 
                if value >= date(2021, 1, 1) and value < date(2021, 2, 1)}

entr_dict = {key: value for key, value in entries_id_dict.items() 
                if value >= cohort_start_date + timedelta(days=30)}

entries_qty = sum(
    1 for k, v in cohort_dict.items()
    if entr_dict.get(k, v) >= v + timedelta(days=30)
)

rolling_retention = round((entries_qty/len(cohort_dict))*100, 5)

**Проверка**

In [127]:
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Lifetime по всем пользователям, посчитанный как интеграл от n-day retention

In [163]:
total = sum(len(v) for v in entries_date_dict.values())
lifetime = round((total/len(registration_id_dict)), 5)

**Проверка**

In [164]:
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Churn rate 29 дня (в долях), посчитанный по всем пользователям